In [ ]:
# Suprress warnings
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # Standard imports
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import numpy as np

    # Drake imports
    from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
    import pydrake
    from pydrake.all import LogOutput, DirectCollocation, DirectTranscription, MathematicalProgram, InputPortSelection

# Imports of other project files
from log_wrapper import LogWrapper
import constants
import finger
import pedestal
from paper import Paper

from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd)

# Other imports
import importlib

In [ ]:
# Matplotlib configuring
plt.style.use(['science', 'no-latex'])
font = {'size'   : 14}
matplotlib.rc('font', **font)

In [ ]:
# Meshcat init
proc, zmq_url, web_url = start_zmq_server_as_subprocess()

In [ ]:
## Pre-finalize steps
builder = pydrake.systems.framework.DiagramBuilder()

# Add all elements
plant, scene_graph = pydrake.multibody.plant.AddMultibodyPlantSceneGraph(builder, time_step=constants.DT)
plant.set_stiction_tolerance(1)
pedestal_instance = pedestal.AddPedestal(plant)

# These joint angles start the paper approximately the right spot, no matter how many links are used
num_links = 2
# def_joint_angles = list(np.zeros(num_links))
# def_joint_angles[0] = np.pi/100
# def_joint_angles[1] = np.pi/100
# def_joint_angles[int(num_links*0.65)-2] = -np.pi/10
# def_joint_angles[int(num_links*0.65)-1] = -np.pi/10
# def_joint_angles[int(num_links*0.65)] = -np.pi/10
# def_joint_angles[int(num_links*0.65)+1] = -np.pi/10
# # def_joint_angles[int(num_links*0.65)+2] = -np.pi/10
def_joint_angles = np.pi/3

paper = Paper(plant, scene_graph, num_links, "NATURAL", default_joint_angle=def_joint_angles, stiffness=2.5e-3, damping=7.12547340446979e-06)
paper.weld_paper_edge(pedestal.PEDESTAL_WIDTH, pedestal.PEDESTAL_HEIGHT)

finger_instance = finger.AddFinger(plant, constants.INIT_Y, constants.INIT_Z)

# Set up logger (needs to happen after all bodies are added)
log_wrapper = LogWrapper(plant.num_bodies())
builder.AddSystem(log_wrapper)

Calculate link inertia to use in damping calculations:

In [ ]:
paper.plant.GetBodyByName("paper_body", paper.get_free_edge_instance()).default_rotational_inertia().CalcPrincipalMomentsOfInertia()[0]

In [ ]:
## CHOOSE CONTROL SYSTEM HERE BY UNCOMMENTING
# # PD control: hits too low
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.05],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )

# PD control: hits too high
# finger_ctrlr = finger.PDFinger(
#     plant,
#     int(finger_instance),
#     [
#         [constants.INIT_Y, constants.INIT_Z],
#         [constants.INIT_Y*1.1, constants.INIT_Z],
#         [constants.INIT_Y*1.1, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2+0.08],
#         [0, pedestal.PEDESTAL_HEIGHT+constants.FINGER_RADIUS/2],
#     ],
#     tspan_per_segment=1,
#     ky=10,
#     kz=10
# )


# Edge feedback
finger_ctrlr = finger.EdgeController(
    plant,
    paper,
    int(finger_instance),
    F_Nd=0.03,
    debug=True,
)

# # Optimization controller
# finger_ctrlr = finger.OptimizationController(
#     plant,
#     paper,
#     int(finger_instance),
#     paper.get_free_edge_instance()
# )

# Blank controller
# finger_ctrlr = finger.BlankController(
#     plant,
#     int(finger_instance)
# )

In [ ]:
plant.Finalize()

## Post finalize steps
# Conect finger controller
builder.AddSystem(finger_ctrlr)
builder.Connect(finger_ctrlr.get_output_port(), plant.get_actuation_input_port(finger_instance))
builder.Connect(plant.get_body_poses_output_port(), finger_ctrlr.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), finger_ctrlr.get_input_port(1))

# Add logger
builder.Connect(plant.get_body_poses_output_port(), log_wrapper.get_input_port(0))
builder.Connect(plant.get_body_spatial_velocities_output_port(), log_wrapper.get_input_port(1))

# Visualization and logging
logger = LogOutput(log_wrapper.get_output_port(), builder)
vis = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder, scene_graph)

# Build diagram and do actions requiring 
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()

In [ ]:
if type(finger_ctrlr) is finger.OptimizationController:
    finger_ctrlr.optimize(plant.GetPositions(diagram_context))

In [ ]:
# Finalize simulation and visualization
simulator = pydrake.systems.analysis.Simulator(diagram, diagram_context)
simulator.Initialize()
vis.start_recording()
simulator.AdvanceTo(constants.TSPAN)
vis.stop_recording()
vis.publish_recording()

In [ ]:
# Plot manipulator position vs. trajectory
if type(finger_ctrlr) is finger.PDFinger:
    plt.figure(figsize=(2*3,2*2))
    plt.plot(logger.data()[12*int(finger_instance)+1],
             logger.data()[12*int(finger_instance)+2],
             label='Manipulator position')
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, label='Trajectory')
    plt.xlabel("$y$ position")
    plt.ylabel("$z$ position")
    plt.legend()
    plt.show()

In [ ]:
# Generate plots for paper
nb = plant.num_bodies()
paper_idxs = [int(i) for i in paper.link_instances]
y_traces = []
z_traces = []
for b in paper_idxs:
    y_traces.append(logger.data()[12*b+1])
    z_traces.append(logger.data()[12*b+2])
y_traces = np.array(y_traces)
z_traces = np.array(z_traces)

times = np.arange(0,constants.TSPAN,constants.TSPAN/10)
cmap = cm.get_cmap("viridis_r")
plt.figure(figsize=(2*3,2*2))
if type(finger_ctrlr) is finger.PDFinger:
    plt.plot(finger_ctrlr.ys, finger_ctrlr.zs, '--k', zorder=-1)
for t in times:
    c = cmap(t/constants.TSPAN)
    idx = int(t/constants.DT)
    plt.plot(y_traces[:-1,idx], z_traces[:-1,idx], color=c)
    plt.scatter(logger.data()[12*int(finger_instance)+1,idx], 
                logger.data()[12*int(finger_instance)+2,idx],
                color=c, s=300, zorder=1)

xlim = plt.xlim()
ylim = plt.ylim()
plt.scatter([xlim[0]-50, xlim[0]-50], [ylim[0]-50, ylim[0]-50], c=[0, constants.TSPAN], cmap=cmap)
plt.xlim(xlim)
plt.ylim(ylim)
cb = plt.colorbar()
cb.set_label("Time")
plt.xlabel("$y$ position")
plt.ylabel("$z$ position")
plt.show()

This is a sanity check to see how much force we're neglecting by ignoring drag. The colored lines are the drag forces at each link, and the dashed black line is the force due to gravity for scale.

Note that the formula for drag force is $\rho v^2 C_d A_{paper}$.

$C_d$ comes from [this site](https://www.engineersedge.com/fluid_flow/rectangular_flat_plate_drag_14036.htm), which says it depends on $L/d$. For 20 links, $L/d\approx15$. Rounding this down to 10, we get $C_d=1.22$.

In [ ]:
# Plot drag forces
plt.figure(figsize=(16, 12))
nb = plant.num_bodies()
paper_idxs = [int(i) for i in paper.link_instances]
vel_traces = []
for b in paper_idxs:
    rho = 1.225
    C_d = paper.link_width*paper.depth
    vel_sqred = logger.data()[12*b+7]**2+logger.data()[12*b+8]**2
    f = 0.5*rho*C_d*vel_sqred
    plt.plot(f)

plt.axhline(paper.link_mass*9.81, color='black', linestyle='--')
plt.show()